# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,24.80,79,21,9.46,GS,1691521344
1,1,porto novo,6.4965,2.6036,78.62,85,12,9.33,BJ,1691521349
2,2,lihue,21.9789,-159.3672,81.99,72,100,21.85,US,1691521349
3,3,mubende,0.5606,31.3900,67.78,64,68,1.54,UG,1691521349
4,4,chibougamau,49.9168,-74.3659,60.42,88,100,8.05,CA,1691521349


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
# Configure the map plot using city_data_df
fig = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    size = "Humidity",
    geo = True,
    colormap="viridis",
    tiles = "OSM",
    colorbar=True,
    hover_cols=["City"],
    title="City Humidity",
    height=500,
    width=800
)

# Display the map
fig


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [52]:
# Narrow down cities that fit criteria and drop any results with null values
# Criteria: Low humidity, medium temperature, medium cloudiness

# Find summary statistics for humidity, temperature, and cloudiness, add to a dataframe
summ_stats_df = city_data_df[['Humidity', 'Max Temp', 'Cloudiness']].describe()

#Ideal weather conditions for me would be:
#Max temp is below average
#Within one std_dev of mean from cloudiness
#Between 25% and 50% quartiles for humidity

# Narrow down the cities that fit these weather conditions
# Start by creating a new dataframe that fits these weather criteria
# Do this using the measuresments in the summary statistics dataframe (summ_stats_df) and the comments above
ideal_weather_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] <= (summ_stats_df.loc['mean', 'Max Temp'])) &
    (city_data_df['Cloudiness'].between(summ_stats_df.loc['mean', 'Cloudiness'] - summ_stats_df.loc['std', 'Cloudiness'], summ_stats_df.loc['mean', 'Cloudiness'] + summ_stats_df.loc['std', 'Cloudiness'])) &
    (city_data_df['Humidity'].between(summ_stats_df.loc['25%', 'Humidity'], summ_stats_df.loc['50%', 'Humidity']))
].copy()

# Drop any rows with null values
ideal_weather_cities_df.dropna(inplace=True)

# Count and print amount of cities in new dataframe
print(f"There are {len(ideal_weather_cities_df)} cities that fit the ideal weather conditions.")

# Display sample data
ideal_weather_cities_df.head()


There are 33 cities that fit the ideal weather conditions.


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,mubende,0.5606,31.3900,67.78,64,68,1.54,UG,1691521349
6,6,coquimbo,-29.9533,-71.3436,62.33,72,75,14.97,CL,1691521245
50,50,nuuk,64.1835,-51.7216,48.81,76,20,5.99,GL,1691521357
61,61,dudinka,69.4058,86.1778,61.63,68,43,1.34,RU,1691521359
70,70,aasiaat,68.7098,-52.8699,39.65,70,19,4.16,GL,1691521360


### Step 3: Create a new DataFrame called `hotel_df`.

In [73]:
# Create a new DataFrame with only the desired columns
hotel_df = ideal_weather_cities_df.loc[:, ['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()



33


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [74]:
# Set parameters to search for a hotel
radius = 10000
limit = 200
categories = "accommodation.hotel"
params = {
    "limit":limit,
    "api_key": geoapify_key,
    "categories": categories
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lng = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make API request using the requests library
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    response_json = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = f"No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mubende - nearest hotel: Hotel Nakayima
coquimbo - nearest hotel: Hotel Iberia
nuuk - nearest hotel: Nordbo
dudinka - nearest hotel: Виктория
aasiaat - nearest hotel: SØMA
stanley - nearest hotel: Hotel 52
mirnyy - nearest hotel: Зарница
keflavik - nearest hotel: Núpan Deluxe
colonia - nearest hotel: Wasserturm Hotel Cologne
talnakh - nearest hotel: Талнах
okhotsk - nearest hotel: No hotel found
vilyuchinsk - nearest hotel: Победа
sake - nearest hotel: No hotel found
aykhal - nearest hotel: Уют
smithers - nearest hotel: Sunshine Inn Hotel
monduli - nearest hotel: No hotel found
constantia - nearest hotel: Hotel Chérica
arno - nearest hotel: Pensionat Grisslehamns
ust-nera - nearest hotel: гостиница "Солнечная"
mirny - nearest hotel: Зарница
tosno - nearest hotel: Комфорт
urengoy - nearest hotel: Геолог
la ronge - nearest hotel: La Ronge Hotel and Suites
taltal - nearest hotel: Hostería Taltal
susuman - nearest hotel: No hotel found
happy valley-goose bay - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
3,mubende,UG,0.5606,31.3900,64,Hotel Nakayima
6,coquimbo,CL,-29.9533,-71.3436,72,Hotel Iberia
50,nuuk,GL,64.1835,-51.7216,76,Nordbo
61,dudinka,RU,69.4058,86.1778,68,Виктория
70,aasiaat,GL,68.7098,-52.8699,70,SØMA
82,stanley,GB,54.8680,-1.6985,59,Hotel 52
84,mirnyy,RU,62.5353,113.9611,59,Зарница
87,keflavik,IS,64.0049,-22.5624,76,Núpan Deluxe
114,colonia,DE,50.9333,6.9500,66,Wasserturm Hotel Cologne
115,talnakh,RU,69.4865,88.3972,65,Талнах


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [75]:
# Make a copy of the hotel_df DataFrame to use for the hotel_map that doesn't have any hotel_map['Hotel Name'] = "Hotel Not Found" values
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != "No hotel found"].copy()

# Configure the map plot
hotel_map = clean_hotel_df.hvplot.points(
    x="Lng", 
    y="Lat", 
    geo = True,
    tiles="OSM", 
    hover_cols=["Hotel Name", "City", "Country"], 
    scale = 2,
    dynamic= True, 
    frame_width = 800,
    frame_height = 600,
    title="Hotels in Cities with Ideal Weather Conditions",
    color = "City"
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)